In [ ]:
import yfinance as yf
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

lock = threading.Lock()

def process_symbol(symbol):
    results = []
    plot_data = None
    try:
        stock_symbol = symbol + ".NS"  # Add .NS to the symbol for NSE
        stock = yf.Ticker(stock_symbol)

        data = stock.history(period="200d")
        if data.empty:
            results.append(f"{symbol}: No data available for the past 200 days.")
            return results, plot_data

        dates = data.index[-50:]
        data = data.drop(columns=['Dividends', 'Stock Splits'])
        MA200 = data['Close'].sum() / 200
        Volume = data['Volume']
        Volume.to_csv('c.csv')

        data = stock.history(period="100d")
        if data.empty:
            results.append(f"{symbol}: No data available for the past 100 days.")
            return results, plot_data

        data = data.drop(columns=['Dividends', 'Stock Splits'])
        MA100 = data['Close'].sum() / 100

        data = stock.history(period="50d")
        if data.empty:
            results.append(f"{symbol}: No data available for the past 50 days.")
            return results, plot_data

        data = data.tail(50)
        data = data.drop(columns=['Dividends', 'Stock Splits'])
        MA50 = data['Close'].sum() / 50

        data = stock.history(period="1d")
        if data.empty:
            results.append(f"{symbol}: No data available for the past 1 day.")
            return results, plot_data

        current_price = data['Close'].iloc[-1]

        if (MA200 >= current_price * 0.99) and (MA200 <= current_price * 1.01):
            results.append(f"{symbol}: The 200-day MA is matching, Current Price: {current_price:.2f}, 200-MA: {MA200:.2f}")
            results.append(f"{symbol}: The 100-day MA is : {MA100:.2f}")
            results.append(f"{symbol}: The 50-day MA is : {MA50:.2f}")
            rsi = calculate_rsi(stock)
            results.append(f"{symbol}: Current RSI: {rsi}")
            plot_data = (symbol, dates, Volume)

        if (MA100 >= current_price * 0.99) and (MA100 <= current_price * 1.01):
            results.append(f"{symbol}: The 100-day MA is matching, Current Price: {current_price:.2f}, 100-MA: {MA100:.2f}")
            results.append(f"{symbol}: The 200-day MA is : {MA200:.2f}")
            results.append(f"{symbol}: The 50-day MA is : {MA50:.2f}")
            rsi = calculate_rsi(stock)
            results.append(f"{symbol}: Current RSI: {rsi}")
            plot_data = (symbol, dates, Volume)

        if (MA50 >= current_price * 0.99) and (MA50 <= current_price * 1.01):
            results.append(f"{symbol}: The 50-day MA is matching, Current Price: {current_price:.2f}, 50-MA: {MA50:.2f}")
            results.append(f"{symbol}: The 100-day MA is : {MA100:.2f}")
            results.append(f"{symbol}: The 200-day MA is : {MA200:.2f}")
            rsi = calculate_rsi(stock)
            results.append(f"{symbol}: Current RSI: {rsi}")
            plot_data = (symbol, dates, Volume)
    except Exception as e:
        results.append(f"An error occurred for {symbol}: {str(e)}")

    return results, plot_data

def calculate_rsi(stock):
    data = stock.history(period="14d")
    if data.empty:
        return "N/A"
    delta = data['Close'].diff().fillna(0)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.sum() / (gain > 0).sum() if (gain > 0).sum() != 0 else 0
    avg_loss = loss.sum() / (loss > 0).sum() if (loss > 0).sum() != 0 else 0
    rs = avg_gain / avg_loss if avg_loss != 0 else 0
    rsi = 100 - (100 / (1 + rs)) if rs != 0 else 0
    return rsi

def plot_volume(symbol, dates, Volume):
    Volume_array = Volume.tail(50).values
    window_size = 5
    moving_average = np.convolve(Volume_array, np.ones(window_size) / window_size, mode='valid')

    plt.figure(figsize=(12, 6))
    plt.bar(dates, Volume_array / 1e7, label=f'{symbol} - 50 Days Volume (in Crores)')
    plt.plot(dates[window_size - 1:], moving_average / 1e7, label='Moving Average', color='orange', linewidth=2)

    plt.xlabel('Date')
    plt.ylabel('Volume')
    plt.title(f'{symbol} - 50 Days Volume')
    plt.xticks(rotation=45, ha="right")
    plt.yticks(np.arange(0, max(Volume_array) / 1e7 + 1, 2))
    plt.legend()
    plt.tight_layout()

    plt.show()
    plt.close()

def get_stock_data():
    equity = pd.read_csv('Equity.csv')
    top_100_symbols = equity['SYMBOL'][:1000]
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(process_symbol, symbol) for symbol in top_100_symbols]
        for future in as_completed(futures):
            result, plot_data = future.result()
            with lock:
                for line in result:
                    print(line)
                if plot_data:
                    plot_volume(*plot_data)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

if __name__ == "__main__":
    get_stock_data()
